In [16]:
pip install pyspark boto3 pandas 


Note: you may need to restart the kernel to use updated packages.


In [12]:
# !apt-get install dpkg-deb
# !wget https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-j-9.0.0.tar.gz
# !tar -xzf mysql-connector-j-9.0.0.tar.gz


/usr/bin/sh: apt-get: command not found
--2024-09-20 03:11:52--  https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-j-9.0.0.tar.gz
Resolving dev.mysql.com (dev.mysql.com)... 184.27.10.232, 2600:1408:9000:78e::2e31, 2600:1408:9000:79a::2e31
Connecting to dev.mysql.com (dev.mysql.com)|184.27.10.232|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://cdn.mysql.com//Downloads/Connector-J/mysql-connector-j-9.0.0.tar.gz [following]
--2024-09-20 03:11:53--  https://cdn.mysql.com//Downloads/Connector-J/mysql-connector-j-9.0.0.tar.gz
Resolving cdn.mysql.com (cdn.mysql.com)... 184.25.42.21, 2600:1408:c400:1884::1d68, 2600:1408:c400:188d::1d68
Connecting to cdn.mysql.com (cdn.mysql.com)|184.25.42.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4469329 (4.3M) [application/x-tar-gz]
Saving to: ‘mysql-connector-j-9.0.0.tar.gz’

100%[======================================>] 4,469,329   --.-K/s   in 0.02s   

2024

In [15]:
# !mv mysql-connector-j-9.0.0/mysql-connector-j-9.0.0.jar /usr/local/lib/

In [17]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import boto3
from datetime import datetime
from pyspark.sql.functions import *
import pandas as pd
import pandas

In [18]:
s3 = boto3.client('s3')
bucket_name = 'g2-s3-trusted'
bucket_name_destination = 'g2-s3-client'
response = s3.list_objects_v2(Bucket=bucket_name)
nomes_sensores = []
latest = datetime.strptime(response['Contents'][0]['Key'][6:32], "%Y-%m-%d %H:%M:%S.%f")

for nome in response['Contents']:
    actual_file_date = datetime.strptime(nome['Key'][6:32], "%Y-%m-%d %H:%M:%S.%f")
    print(nome['Key'][6:32])
    if actual_file_date > latest:
        latest = actual_file_date
        
print(f"a mais nova é {latest}")

/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


2024-03-03 22:00:00.335932
2024-03-11 13:00:00.101344
a mais nova é 2024-03-11 13:00:00.101344


In [19]:
conf = SparkConf()
conf.set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.900")
conf.set("spark.jars", "/usr/local/lib/mysql-connector-j-9.0.0.jar")
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
conf.set("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print(f'until_{latest.strftime("%Y-%m-%d %H:%M:%S.%f")}.csv')

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-352955a9-e081-491f-abf9-e9a35f2c9bb2;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 545ms :: artifacts dl 36ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.900 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

until_2024-03-11 13:00:00.101344.csv


In [20]:
data = spark.read.option('delimiter', ',').option('header', 'true').csv(f's3a://{bucket_name}/until_{latest.strftime("%Y-%m-%d %H:%M:%S.%f")}.csv')

24/09/20 03:15:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [21]:
data.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- gydci: string (nullable = true)
 |-- gyneo: string (nullable = true)
 |-- keyestudio: string (nullable = true)
 |-- mq4_buthane: string (nullable = true)
 |-- mq4_methane: string (nullable = true)
 |-- proteus: string (nullable = true)



In [22]:
data.show()

+--------------------+-----+--------------------+----------+-----------+-----------+----------+
|           timestamp|gydci|               gyneo|keyestudio|mq4_buthane|mq4_methane|   proteus|
+--------------------+-----+--------------------+----------+-----------+-----------+----------+
|2024-03-01 01:00:...|  0.0|                   0|         0|          0|    1963.95|1895.71015|
|2024-03-01 01:00:...|  0.0|                   0|         0|          0|      869.7| 2748.6147|
|2024-03-01 02:00:...|  0.0|                   0|         0|          0|    4876.39|2433.83384|
|2024-03-01 02:00:...|  0.0|                   0|         0|          0|    3777.88|1949.84186|
|2024-03-01 03:00:...| 40.1|                   0|         0|          0|    3809.04|2399.63643|
|2024-03-01 03:00:...| 42.0|                   0|         0|          0|    4770.14|2844.21008|
|2024-03-01 04:00:...| 42.0|{'lat': -6.293392...|         0|          0|    2867.06|2162.27918|
|2024-03-01 04:00:...| 42.0|{'lat': -6.2

In [23]:
data_corrected = data.withColumn("keyestudio", col("keyestudio").cast("integer")) \
                 .withColumn("timestamp", to_timestamp("timestamp", "yyyy-MM-dd HH:mm:ss.SSSSSS")) \
                 .withColumn("mq4_buthane", col("mq4_buthane").cast("double")) \
                 .withColumn("gydci", col("gydci").cast("double")) \
                 .withColumn("mq4_methane", col("mq4_methane").cast("double")) \
                 .withColumn("proteus", col("proteus").cast("double")) \


In [24]:
data_corrected.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- gydci: double (nullable = true)
 |-- gyneo: string (nullable = true)
 |-- keyestudio: integer (nullable = true)
 |-- mq4_buthane: double (nullable = true)
 |-- mq4_methane: double (nullable = true)
 |-- proteus: double (nullable = true)



In [25]:
data_corrected.show()

+--------------------+-----+--------------------+----------+-----------+-----------+----------+
|           timestamp|gydci|               gyneo|keyestudio|mq4_buthane|mq4_methane|   proteus|
+--------------------+-----+--------------------+----------+-----------+-----------+----------+
|2024-03-01 01:00:...|  0.0|                   0|         0|        0.0|    1963.95|1895.71015|
|2024-03-01 01:00:...|  0.0|                   0|         0|        0.0|      869.7| 2748.6147|
|2024-03-01 02:00:...|  0.0|                   0|         0|        0.0|    4876.39|2433.83384|
|2024-03-01 02:00:...|  0.0|                   0|         0|        0.0|    3777.88|1949.84186|
|2024-03-01 03:00:...| 40.1|                   0|         0|        0.0|    3809.04|2399.63643|
|2024-03-01 03:00:...| 42.0|                   0|         0|        0.0|    4770.14|2844.21008|
|2024-03-01 04:00:...| 42.0|{'lat': -6.293392...|         0|        0.0|    2867.06|2162.27918|
|2024-03-01 04:00:...| 42.0|{'lat': -6.2

In [26]:
data_corrected.write.csv(f's3a://g2-s3-client/values/complete_datasets/until_{latest}', mode='overwrite')

24/09/20 03:15:32 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
24/09/20 03:15:33 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [27]:
data.select(get_json_object(data.gyneo, '$.lat'), get_json_object(data.gyneo, '$.long')).show()

+-----------------------------+------------------------------+
|get_json_object(gyneo, $.lat)|get_json_object(gyneo, $.long)|
+-----------------------------+------------------------------+
|                         null|                          null|
|                         null|                          null|
|                         null|                          null|
|                         null|                          null|
|                         null|                          null|
|                         null|                          null|
|           -6.293392349522247|             -51.5043530131051|
|           -6.279693478046926|           -51.588349679569625|
|           -6.279693478046926|           -51.588349679569625|
|           -6.279693478046926|           -51.588349679569625|
|           -6.279693478046926|           -51.588349679569625|
|           -6.279693478046926|           -51.588349679569625|
|           -6.279693478046926|           -51.588349679

In [28]:
data_corrected = data_corrected.withColumn('suspicious_water', when((col('proteus') > 10000) & (col('keyestudio')>600), True).otherwise(False))

In [30]:
#criar agrupamentos
#criar colunas de classificação
#criar dataset para geolocalização

+--------------------+-----+--------------------+----------+-----------+-----------+----------+----------------+
|           timestamp|gydci|               gyneo|keyestudio|mq4_buthane|mq4_methane|   proteus|suspicious_water|
+--------------------+-----+--------------------+----------+-----------+-----------+----------+----------------+
|2024-03-01 01:00:...|  0.0|                   0|         0|        0.0|    1963.95|1895.71015|           false|
|2024-03-01 01:00:...|  0.0|                   0|         0|        0.0|      869.7| 2748.6147|           false|
|2024-03-01 02:00:...|  0.0|                   0|         0|        0.0|    4876.39|2433.83384|           false|
|2024-03-01 02:00:...|  0.0|                   0|         0|        0.0|    3777.88|1949.84186|           false|
|2024-03-01 03:00:...| 40.1|                   0|         0|        0.0|    3809.04|2399.63643|           false|
|2024-03-01 03:00:...| 42.0|                   0|         0|        0.0|    4770.14|2844.21008| 

24/09/20 03:17:15 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
24/09/20 03:17:16 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [32]:
jdbc_url = "jdbc:mysql://52.87.180.211:3306/cattlecare"
table_name = "values_sensors"
connection_properties = {
    "user": "root",           
    "password": "toor_db",  
    "driver": "com.mysql.cj.jdbc.Driver"
}
data_corrected.write.jdbc(url=jdbc_url, table=table_name, mode="overwrite", properties=connection_properties)